## 准备数据

In [5]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [6]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [38]:
class myModel:
    def __init__(self):
        input_dim = 28 * 28
        hidden_dim1 = 512
        hidden_dim2 = 256
        hidden_dim3 = 128
        hidden_dim4 = 64
        output_dim = 10

        initializer = tf.keras.initializers.HeNormal()

        self.W1 = tf.Variable(initializer([input_dim, hidden_dim1]), dtype=tf.float32)
        self.b1 = tf.Variable(tf.zeros([hidden_dim1]), dtype=tf.float32)

        self.W2 = tf.Variable(initializer([hidden_dim1, hidden_dim2]), dtype=tf.float32)
        self.b2 = tf.Variable(tf.zeros([hidden_dim2]), dtype=tf.float32)

        self.W3 = tf.Variable(initializer([hidden_dim2, hidden_dim3]), dtype=tf.float32)
        self.b3 = tf.Variable(tf.zeros([hidden_dim3]), dtype=tf.float32)

        self.W4 = tf.Variable(initializer([hidden_dim3, hidden_dim4]), dtype=tf.float32)
        self.b4 = tf.Variable(tf.zeros([hidden_dim4]), dtype=tf.float32)

        self.W5 = tf.Variable(initializer([hidden_dim4, output_dim]), dtype=tf.float32)
        self.b5 = tf.Variable(tf.zeros([output_dim]), dtype=tf.float32)

    def __call__(self, x):
        x = tf.reshape(x, [-1, 28 * 28])  # 展平输入
        # 第一层
        z1 = tf.matmul(x, self.W1) + self.b1
        a1 = tf.nn.relu6(z1)

        # 第二层
        z2 = tf.matmul(a1, self.W2) + self.b2
        a2 = tf.nn.relu6(z2)

        # 第三层
        z3 = tf.matmul(a2, self.W3) + self.b3
        a3 = tf.nn.relu6(z3)

        # 第四层
        z4 = tf.matmul(a3, self.W4) + self.b4
        a4 = tf.nn.relu6(z4)

        # 输出层 logits
        logits = tf.matmul(a4, self.W5) + self.b5
        return logits

## 计算 loss

In [31]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [40]:
    
model = myModel()
optimizer = optimizers.Adam()

train_data, test_data = mnist_dataset()
for epoch in range(100):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())

loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))
print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.4112074 ; accuracy 0.08975
epoch 1 : loss 2.3894746 ; accuracy 0.094283335
epoch 2 : loss 2.3694115 ; accuracy 0.09915
epoch 3 : loss 2.3508036 ; accuracy 0.10365
epoch 4 : loss 2.3334687 ; accuracy 0.108616665
epoch 5 : loss 2.3172765 ; accuracy 0.11333334
epoch 6 : loss 2.302075 ; accuracy 0.11855
epoch 7 : loss 2.2876954 ; accuracy 0.12408333
epoch 8 : loss 2.274041 ; accuracy 0.12995
epoch 9 : loss 2.2610188 ; accuracy 0.1359
epoch 10 : loss 2.2485385 ; accuracy 0.14233333
epoch 11 : loss 2.236551 ; accuracy 0.14983334
epoch 12 : loss 2.2249782 ; accuracy 0.15831667
epoch 13 : loss 2.2137635 ; accuracy 0.16771667
epoch 14 : loss 2.2028835 ; accuracy 0.1771
epoch 15 : loss 2.1922843 ; accuracy 0.18746667
epoch 16 : loss 2.1819222 ; accuracy 0.19806667
epoch 17 : loss 2.1717842 ; accuracy 0.20786667
epoch 18 : loss 2.1618383 ; accuracy 0.21818334
epoch 19 : loss 2.152078 ; accuracy 0.22878334
epoch 20 : loss 2.1424794 ; accuracy 0.23785
epoch 21 : loss 2.1330094 ; ac